# Q1. Install MLflow

In [3]:
!mlflow --version

mlflow, version 2.20.1


# Q2. Download and preprocess the data

In [4]:
!python preprocess_data.py --raw_data_path ./rawdata --dest_path ./output

# Q3. Train a model with autolog

In [11]:
!python train.py --data_path output

2025/02/04 19:51:37 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment-hw2' does not exist. Creating a new experiment.
2025/02/04 19:51:39 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/02/04 19:51:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpqd0r2srl/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 


In [20]:
import mlflow
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name('nyc-taxi-experiment-hw2')
runs = client.search_runs(experiment_ids=[experiment.experiment_id])

print(runs[0].data.params.get('min_samples_split'))

2


# Q4. Launch the tracking server locally

In [21]:
!mlflow server --backend-store-uri "sqlite:///mlflow.db" --default-artifact-root "./artifacts"

2025/02/04 20:04:23 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/02/04 20:04:23 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

# Q5. Tune model hyperparameters

In [23]:
!python hpo.py

2025/02/04 20:11:35 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
🏃 View run zealous-bat-753 at: http://127.0.0.1:5000/#/experiments/1/runs/0d7d87df9ea4455a95f986322db6e6a8

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run resilient-bird-592 at: http://127.0.0.1:5000/#/experiments/1/runs/938789eeee1f4e66bb7c256003a417a2

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run selective-shark-884 at: http://127.0.0.1:5000/#/experiments/1/runs/fa934270d969461cb9ed6c34cb602587

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run valuable-dog-400 at: http://127.0.0.1:5000/#/experiments/1/runs/888bf9d7c8e84dcaad4fac2950de563e

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run glamorous-hound-104 at: http://127.0.0.1:5000/#/experiments/1/runs/b976168d150841088827

In [26]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.get_experiment_by_name("random-forest-hyperopt")
client = mlflow.tracking.MlflowClient()

best_run = client.search_runs(
  experiment_ids = experiment.experiment_id, 
  run_view_type= mlflow.entities.ViewType.ACTIVE_ONLY,
  max_results=1,
  order_by=["metrics.rmse ASC"]
)[0]
print('The best validation rmse is :', round(best_run.data.metrics['rmse'], 3))

The best validation rmse is : 5.335


# Q6. Promote the best model to the model registry

In [27]:
!python register_model.py

2025/02/04 20:22:52 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2025/02/04 20:22:52 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/02/04 20:22:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpxgbz9t1x/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
🏃 View run clumsy-goose-75 at: http://127.0.0.1:5000/#/experiments/2/runs/c5bf786711404d83bcd4cf644674eaaf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
2025/02/04 20:23:03 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/02/04 20:23:21 WARNING mlflow.utils.environment: Enc